In [1]:
import pandas as pd
# from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBRegressor
from sklearn.tree import  plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import KFold

concrete = pd.read_csv("Concrete_Data.csv")
X = concrete.drop('Strength',axis=1)
y = concrete[['Strength']]
kfold =KFold(n_splits=5, shuffle=True, random_state=23)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=23)

With Passthrough

In [11]:
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, StandardScaler
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor


rf =RandomForestRegressor(random_state=23)
knn = KNeighborsRegressor()
elastic = ElasticNet()
catboost = CatBoostRegressor(random_state=23)
dtr = DecisionTreeRegressor(random_state=23)
lgbm = LGBMRegressor(random_state=23)
gbm = GradientBoostingRegressor(random_state=23)
scaler = StandardScaler()
pipe = Pipeline([('SCL',scaler),('KNN',knn)])
stack = StackingRegressor([('ELASTIC',elastic),('PIPE',pipe),('TREE',dtr)],final_estimator=gbm, passthrough=True)
stack.fit(X_train, y_train) #Executes all ops on train set
y_pred = stack.predict(X_test) #Executes all ops on test set
print(r2_score(y_test, y_pred))

0.8986688812502333


In [8]:
print(stack.get_params())

{'cv': None, 'estimators': [('elastic', ElasticNet()), ('PIPE', Pipeline(steps=[('SCL', StandardScaler()), ('KNN', KNeighborsRegressor())])), ('DTC', DecisionTreeRegressor(random_state=23))], 'final_estimator__alpha': 0.9, 'final_estimator__ccp_alpha': 0.0, 'final_estimator__criterion': 'friedman_mse', 'final_estimator__init': None, 'final_estimator__learning_rate': 0.1, 'final_estimator__loss': 'squared_error', 'final_estimator__max_depth': 3, 'final_estimator__max_features': None, 'final_estimator__max_leaf_nodes': None, 'final_estimator__min_impurity_decrease': 0.0, 'final_estimator__min_samples_leaf': 1, 'final_estimator__min_samples_split': 2, 'final_estimator__min_weight_fraction_leaf': 0.0, 'final_estimator__n_estimators': 100, 'final_estimator__n_iter_no_change': None, 'final_estimator__random_state': 23, 'final_estimator__subsample': 1.0, 'final_estimator__tol': 0.0001, 'final_estimator__validation_fraction': 0.1, 'final_estimator__verbose': 0, 'final_estimator__warm_start': F

In [12]:
from sklearn.model_selection import RandomizedSearchCV


params = {'ELASTIC__l1_ratio':np.linspace(0.001,0.99,4),'ELASTIC__alpha':[0.1,1],'PIPE__KNN__n_neighbors':[1,5,10],
          'TREE__min_samples_leaf':[1,5,10],'TREE__min_samples_split':[2,5,10],'final_estimator':[gbm,rf,lgbm,catboost]}
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
gcv = RandomizedSearchCV(stack, param_distributions=params, cv=kfold, n_jobs=4, verbose=3)
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Learning rate set to 0.03971
0:	learn: 16.3934817	total: 54.2ms	remaining: 54.2s
1:	learn: 15.9860451	total: 55.2ms	remaining: 27.6s
2:	learn: 15.5843082	total: 56.6ms	remaining: 18.8s
3:	learn: 15.2254143	total: 58.6ms	remaining: 14.6s
4:	learn: 14.8357629	total: 66.9ms	remaining: 13.3s
5:	learn: 14.4954612	total: 68.7ms	remaining: 11.4s
6:	learn: 14.1836221	total: 70.1ms	remaining: 9.94s
7:	learn: 13.8738798	total: 72.6ms	remaining: 9s
8:	learn: 13.5210739	total: 75.7ms	remaining: 8.33s
9:	learn: 13.2397901	total: 77.7ms	remaining: 7.69s
10:	learn: 12.9426009	total: 79.1ms	remaining: 7.11s
11:	learn: 12.6483084	total: 84ms	remaining: 6.92s
12:	learn: 12.3789708	total: 98.6ms	remaining: 7.49s
13:	learn: 12.1120504	total: 119ms	remaining: 8.35s
14:	learn: 11.8741684	total: 121ms	remaining: 7.93s
15:	learn: 11.6438738	total: 122ms	remaining: 7.52s
16:	learn: 11.4245479	total: 143ms	remaining: 8.26s
17:	learn: 11.2041738	total: 155ms	remaining: 8.46s
18:	learn: 11.0028275	total: 176ms	re

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


43:	learn: 7.6219420	total: 550ms	remaining: 11.9s
44:	learn: 7.5482053	total: 552ms	remaining: 11.7s
45:	learn: 7.4665819	total: 590ms	remaining: 12.2s
46:	learn: 7.4011461	total: 624ms	remaining: 12.7s
47:	learn: 7.3144417	total: 631ms	remaining: 12.5s
48:	learn: 7.2391185	total: 632ms	remaining: 12.3s
49:	learn: 7.1725288	total: 663ms	remaining: 12.6s
50:	learn: 7.0995363	total: 689ms	remaining: 12.8s
51:	learn: 7.0485690	total: 693ms	remaining: 12.6s
52:	learn: 6.9871922	total: 718ms	remaining: 12.8s
53:	learn: 6.9304329	total: 751ms	remaining: 13.2s
54:	learn: 6.8665057	total: 758ms	remaining: 13s
55:	learn: 6.8091046	total: 762ms	remaining: 12.8s
56:	learn: 6.7655279	total: 796ms	remaining: 13.2s
57:	learn: 6.7254662	total: 800ms	remaining: 13s
58:	learn: 6.6782098	total: 810ms	remaining: 12.9s
59:	learn: 6.6162625	total: 829ms	remaining: 13s
60:	learn: 6.5600970	total: 831ms	remaining: 12.8s
61:	learn: 6.5174588	total: 834ms	remaining: 12.6s
62:	learn: 6.4622786	total: 835ms	rem

...

[CV 1/5] END ELASTIC__alpha=1, ELASTIC__l1_ratio=0.33066666666666666, PIPE__KNN__n_neighbors=5, TREE__min_samples_leaf=5, TREE__min_samples_split=5, final_estimator=RandomForestRegressor(random_state=23);, score=0.896 total time=   6.3s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[CV 2/5] END ELASTIC__alpha=1, ELASTIC__l1_ratio=0.33066666666666666, PIPE__KNN__n_neighbors=5, TREE__min_samples_leaf=5, TREE__min_samples_split=5, final_estimator=RandomForestRegressor(random_state=23);, score=0.890 total time=   6.6s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[CV 5/5] END ELASTIC__alpha=0.1, ELASTIC__l1_ratio=0.001, PIPE__KNN__n_neighbors=1, TREE__min_samples_leaf=10, TREE__min_samples_split=2, final_estimator=LGBMRegressor(random_state=23);, score=0.869 total time=  16.9s
[CV 3/5] END ELASTIC__alpha=1, ELASTIC__l1_ratio=0.33066666666666666, PIPE__KNN__n_neighbors=5, TREE__min_samples_leaf=5, TREE__min_samples_split=5, final_estimator=RandomForestRegressor(random_state=23);, score=0.893 total time=   6.5s
[CV 4/5] END ELASTIC__alpha=1, ELASTIC__l1_ratio=0.33066666666666666, PIPE__KNN__n_neighbors=5, TREE__min_samples_leaf=5, TREE__min_samples_split=5, final_estimator=RandomForestRegressor(random_state=23);, score=0.882 total time=   5.7s
[CV 5/5] END ELASTIC__alpha=1, ELASTIC__l1_ratio=0.33066666666666666, PIPE__KNN__n_neighbors=5, TREE__min_samples_leaf=5, TREE__min_samples_split=5, final_estimator=RandomForestRegressor(random_state=23);, score=0.849 total time=   5

In [13]:
from joblib import dump, load
bm =  gcv.best_estimator_
dump(bm,'StackConcrete.job')

['VotingConcrete.job']

In [11]:
xgbm = XGBRegressor(random_state = 23)
params = {'max_depth':[2, 3, 4, 5, 6, None],'learning_rate': np.linspace(0.001,0.999,10),'n_estimators':[50,100,150]}
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
gcv = RandomizedSearchCV(xgbm, param_distributions=params, cv=kfold, n_jobs=8, random_state=23)
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

TypeError: 'StratifiedKFold' object is not callable

In [14]:
from sklearn.ensemble import RandomForestRegressor


rf =RandomForestRegressor(random_state=23)
params = {'max_features':[3, 4, 5, 6]}
KFold = KFold(n_splits=5, shuffle=True, random_state=23)
gcv = GridSearchCV(rf, param_grid=params, cv=KFold)
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'max_features': 5}
0.9145984795255835


For comparison

In [6]:
from sklearn.tree import DecisionTreeRegressor


dtc =DecisionTreeRegressor(random_state=23)
params = {'max_depth':[3, 4, 5, 6], 'min_samples_split':[2,5,10], 'min_samples_leaf':[1,3,5,7,10,15]}
kfold = KFold(n_splits=5, shuffle=True, random_state=23)
gcv = GridSearchCV(dtc, param_grid=params, cv=kfold)
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2}
0.8468060394889664
